In [1]:
from utils import *

import folium
import geopandas as gpd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

## Data description

In [2]:
macro_df = pd.read_csv('data/macro.csv', parse_dates=['timestamp'])
train_df = pd.read_csv('data/train.csv', index_col='id', parse_dates=['timestamp'])
test_df = pd.read_csv('data/test.csv', index_col='id', parse_dates=['timestamp'])
tverskoe_issue_fix(train_df)
tverskoe_issue_fix(test_df)

Fix:  550
Fix:  149


## 1. Data preprocessing
## I part (encoding and correcting mistakes)

### Macro dataset

In [3]:
macro_df['child_on_acc_pre_school'] = macro_df['child_on_acc_pre_school'].str.replace('#!', 'nan')
for column in macro_df.select_dtypes('object').columns:
    macro_df[column] = macro_df[column].str.replace(',', '.')
    macro_df[column] = macro_df[column].astype(float)

if not len(macro_df.select_dtypes('object').columns):
    print('OK')

OK


### Train dataset

In [4]:
train_df = encode(train_df)

### Test dataset

In [5]:
test_df = encode(test_df)

## II part (Filling missing values)

XGBRegressor model handles `np.NaN` values itself

## 2. Encoding `sub_area` feature

In [6]:
train_df['is_train'] = 1
test_df['is_train'] = 0

coords_df = pd.read_csv('data/coords.csv', index_col='id')
all_df = pd.concat([train_df, test_df])

all_df['latitude'] = coords_df['latitude']
all_df['longitude'] = coords_df['longitude']

## 3. Removing outliers

In [7]:
all_df = remove_outliers(all_df)

In [8]:
train_df = all_df[all_df['is_train'] == 1].drop(['is_train'], axis=1)
test_df = all_df[all_df['is_train'] == 0].drop(['is_train', 'price_doc'], axis=1)

In [ ]:
# train_df = remove_fake_prices(train_df)
idx_outliers = np.loadtxt('data/idx_outliers.txt').astype(int)
train_df = train_df.drop(idx_outliers)

## 4. Modeling

### `product_type == 'Investment'`

In [ ]:
train_df_0 = train_df[train_df['product_type'] == 0]
test_df_0 = test_df[test_df['product_type'] == 0]

X_0 = train_df_0.drop(['sub_area', 'price_doc'], axis=1).copy()
y_0 = np.log1p(train_df_0['price_doc'])
X_test_0 = test_df_0.drop(['sub_area'], axis=1).copy()

In [ ]:
X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
X_train_0.shape, X_val_0.shape

((13405, 292), (3352, 292))

In [ ]:
params = {'objective': 'reg:squarederror',
          'n_estimators': 10000,
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 5,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_0 = XGBRegressor(**params)

model_0.fit(X_train_0, y_train_0, eval_set=[(X_train_0, y_train_0), (X_val_0, y_val_0)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.40691	validation_1-rmse:8.40126
[1]	validation_0-rmse:7.98749	validation_1-rmse:7.98196
[2]	validation_0-rmse:7.58909	validation_1-rmse:7.58359
[3]	validation_0-rmse:7.21059	validation_1-rmse:7.20524
[4]	validation_0-rmse:6.85103	validation_1-rmse:6.84583
[5]	validation_0-rmse:6.50946	validation_1-rmse:6.50440
[6]	validation_0-rmse:6.18498	validation_1-rmse:6.18009
[7]	validation_0-rmse:5.87675	validation_1-rmse:5.87199
[8]	validation_0-rmse:5.58395	validation_1-rmse:5.57935
[9]	validation_0-rmse:5.30581	validation_1-rmse:5.30130
[10]	validation_0-rmse:5.04161	validation_1-rmse:5.03713
[11]	validation_0-rmse:4.79062	validation_1-rmse:4.78634
[12]	validation_0-rmse:4.55223	validation_1-rmse:4.54803
[13]	validation_0-rmse:4.32579	validation_1-rmse:4.32188
[14]	validation_0-rmse:4.11065	validation_1-rmse:4.10686
[15]	validation_0-rmse:3.90628	validation_1-rmse:3.90266
[16]	validation_0-rmse:3.71215	validation_1-rmse:3.70875
[17]	validation_0-rmse:3.52774	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=5, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [ ]:
y_pred_train = np.abs(model_0.predict(X_train_0))
y_pred_val = np.abs(model_0.predict(X_val_0))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train_0), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val_0), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train_0), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val_0), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.10654313497475247
 VALIDATION_1: 0.1401569506454773
___________________________________
      MAE

      TRAIN_1: 602643.8951137636
 VALIDATION_1: 788203.1790721955


### `product_type == 'OwnerOccupier'`

In [ ]:
train_df_1 = train_df[train_df['product_type'] == 1]
test_df_1 = test_df[test_df['product_type'] == 1]

X_1 = train_df_1.drop(['sub_area', 'price_doc'], axis=1).copy()
y_1 = np.log1p(train_df_1['price_doc'])
X_test_1 = test_df_1.drop(['sub_area'], axis=1).copy()

In [ ]:
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)
X_train_1.shape, X_val_1.shape

((8815, 292), (2204, 292))

In [ ]:
params = {'objective': 'reg:squarederror',
          'n_estimators': 10000,
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 5,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_1 = XGBRegressor(**params)

model_1.fit(X_train_1, y_train_1, eval_set=[(X_train_1, y_train_1), (X_val_1, y_val_1)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.14738	validation_1-rmse:8.15221
[1]	validation_0-rmse:7.74130	validation_1-rmse:7.74552
[2]	validation_0-rmse:7.35556	validation_1-rmse:7.35926
[3]	validation_0-rmse:6.98906	validation_1-rmse:6.99297
[4]	validation_0-rmse:6.64088	validation_1-rmse:6.64479
[5]	validation_0-rmse:6.31009	validation_1-rmse:6.31355
[6]	validation_0-rmse:5.99588	validation_1-rmse:5.99948
[7]	validation_0-rmse:5.69734	validation_1-rmse:5.70050
[8]	validation_0-rmse:5.41374	validation_1-rmse:5.41676
[9]	validation_0-rmse:5.14433	validation_1-rmse:5.14738
[10]	validation_0-rmse:4.88837	validation_1-rmse:4.89114
[11]	validation_0-rmse:4.64522	validation_1-rmse:4.64787
[12]	validation_0-rmse:4.41422	validation_1-rmse:4.41674
[13]	validation_0-rmse:4.19481	validation_1-rmse:4.19709
[14]	validation_0-rmse:3.98633	validation_1-rmse:3.98843
[15]	validation_0-rmse:3.78825	validation_1-rmse:3.79035
[16]	validation_0-rmse:3.60011	validation_1-rmse:3.60210
[17]	validation_0-rmse:3.42138	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=5, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [ ]:
y_pred_train = np.abs(model_1.predict(X_train_1))
y_pred_val = np.abs(model_1.predict(X_val_1))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train_1), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val_1), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train_1), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val_1), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.05480818033481479
 VALIDATION_1: 0.0935553285712718
___________________________________
      MAE

      TRAIN_1: 258670.53963414644
 VALIDATION_1: 424848.6755898368


### `StackingRegressor`

In [ ]:
raise StopIteration

In [ ]:
test_df_nan = test_df[test_df['product_type'].isna()]
X_test_nan = test_df_nan.drop(['sub_area'], axis=1).copy()

In [ ]:
submission = pd.read_csv('data/sample_submission.csv', index_col='id')
pred_0 = np.expm1(model_0.predict(X_test_0))
pred_1 = np.expm1(model_1.predict(X_test_1))
pred_nan = np.expm1(model_1.predict(X_test_nan))

if len(pred_0[pred_0 < 0]) or len(pred_1[pred_1 < 0]) or len(pred_nan[pred_nan < 0]):
    print('WARNING: NEGATIVE PREDICTIONS')

In [ ]:
submission.loc[X_test_0.index, 'price_doc'] = 0.87*pred_0
submission.loc[X_test_1.index, 'price_doc'] = 0.93*pred_1
submission.loc[X_test_nan.index, 'price_doc'] = 0.95*pred_nan
submission.to_csv('submits/submission.csv', index='id')

In [ ]:
# !kaggle competitions submit -c sberbank-russian-housing-market -f "submits/submission.csv" -m "XGBRegressor tuned"

100%|████████████████████████████████████████| 121k/121k [00:02<00:00, 61.0kB/s]
Successfully submitted to Sberbank Russian Housing Market